<a href="https://colab.research.google.com/github/AntonioBorz/NeuroEmotions/blob/main/TelegramBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch
import torch.nn as nn
from collections import Counter

In [ ]:
# # Считываем датасет
# n1 = ['ID', 'Rating', 'Date', 'Place Name', 'Text']
# data_positive = pd.read_csv('db/5_4.csv')
# data_negative = pd.read_csv('db/1_2.csv')

# n2 = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
# data_positive2 = pd.read_csv('db/positive.csv', sep=';', names=n2, usecols=['text'])
# data_negative2 = pd.read_csv('db/negative.csv', sep=';', names=n2, usecols=['text'])

In [ ]:
# # Формирование сбалансированного датасета
# sample_size1 = 3000
# reviews_withoutshuffle_1 = np.concatenate((
#     data_positive['Text'].values[:sample_size1],
#     data_negative['Text'].values[:sample_size1]
# ), axis=0)

# labels_withoutshuffle_1 = np.asarray([1] * sample_size1 + [0] * sample_size1)

# assert len(reviews_withoutshuffle_1) == len(labels_withoutshuffle_1)

# sample_size2 = 40000
# reviews_withoutshuffle_2 = np.concatenate((
#     data_positive2['text'].values[:sample_size2],
#     data_negative2['text'].values[:sample_size2]
# ), axis=0)

# labels_withoutshuffle_2 = np.asarray([1] * sample_size2 + [0] * sample_size2)

# assert len(reviews_withoutshuffle_2) == len(labels_withoutshuffle_2)

# all_reviews = np.concatenate((reviews_withoutshuffle_1, reviews_withoutshuffle_2), axis=0)
# all_labels = np.concatenate((labels_withoutshuffle_1, labels_withoutshuffle_2), axis=0)

# reviews, labels = shuffle(all_reviews, all_labels, random_state=0)

# assert len(reviews) == len(labels)

In [ ]:
# # Токенизация
# def tokenize():
#   punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
#   all_reviews = 'separator'.join(reviews)
#   all_reviews = all_reviews.lower()
#   all_text = ''.join([c for c in all_reviews if c not in punctuation])
#   texts_split = all_text.split('separator')
#   all_text = ' '.join(texts_split)
#   words = all_text.split()
#   return words, texts_split

# words, texts_split = tokenize()

In [ ]:
# df = pd.DataFrame(words, columns=['info'])
# df.to_csv("db/words.csv", sep=";", encoding="utf-8")

# df1 = pd.DataFrame(texts_split, columns=['info'])
# df1.to_csv("db/texts_split.csv", sep=";", encoding="utf-8")

In [ ]:
words = pd.read_csv('db/words.csv', sep=";", encoding="utf-8")
words = words['info']

texts_split = pd.read_csv('db/texts_split.csv', sep=";", encoding="utf-8")
texts_split = texts_split['info']

print("Добавили датасет")

Добавили датасет


In [ ]:
# new_reviews = []
# for review in texts_split:
#     review = review.split()
#     new_text = []
#     for word in review:
#         if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
#             new_text.append(word)
#     new_reviews.append(new_text)

In [ ]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii - 1 for ii, word in enumerate(vocab, 1)}
# reviews_ints = []
# for review in new_reviews:
#     reviews_ints.append([vocab_to_int[word] for word in review])

In [ ]:
def add_pads(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        if len(row) == 0:
            continue
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

print("Прошли функцию токенезации")

Прошли функцию токенезации


In [ ]:
# Архитектура нейронной сети
class LSTM_architecture(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, number_of_layers, drop=0.5):
        super(LSTM_architecture, self).__init__()
        self.output_size = output_size
        self.number_of_layers = number_of_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, number_of_layers,dropout=drop, batch_first=True)
        self.dropout = nn.Dropout(0.45)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden_state):
        lstm_out, hidden_state = self.lstm(self.embedding(x.long()), hidden_state)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)
        batch_size = x.size(0)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        return sig_out, hidden_state

    def init_hidden_state(self, batch_size):
        weight = next(self.parameters()).data
        hidden_state = (weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_())
        return hidden_state

In [ ]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 100
hidden_dim = 128
n_layers = 2
model = LSTM_architecture(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.load_state_dict(torch.load('emotionalСoloring.pth', map_location=torch.device('cpu')))

print("Загрузили модель")

Загрузили модель


C:\Users\golun\AppData\Local\Temp\ipykernel_12772\56841439.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('emotionalСoloring.pth', map_

In [ ]:
# Предсказание окраски по тексту
def tokenize_text(test_review):
    test_review = test_review.lower()
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()

    new_text = []
    for word in test_words:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)

    test_ints = []
    # Пропускаем слова, отсутствующие в словаре
    test_ints.append([vocab_to_int[word] for word in new_text if word in vocab_to_int])

    return test_ints


def predict(model, test_review, sequence_length=30):
    model.eval()
    test_ints = tokenize_text(test_review)
    seq_length = sequence_length

    if len(test_ints[0]) == 0:
        return "Не удалось распознать слова из отзыва."

    features = add_pads(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    h = model.init_hidden_state(batch_size)
    output, h = model(feature_tensor, h)

    pred = torch.round(output.squeeze())
    print()

    if (pred.item() == 1):
        return f"Вероятность положительного ответа {output.item():.6f}. Отзыв положительный."
    else:
        return f"Вероятность положительного ответа {output.item():.6f}. Отзыв негативный."

print("Прошли функцию предсказания")

ПРошли функцию предсказания


In [ ]:
import telebot

In [ ]:
def getToken(fileName):
    with open(fileName, 'r') as file:
        return file.read()

In [ ]:
bot = telebot.TeleBot(getToken('Сonfig.ini'))

print("Бот запущен")

@bot.message_handler(commands=['start'])
def main(message):
  bot.send_message(message.chat.id, 'Введите отзыв на русском языке.')

@bot.message_handler()
def info(message):
  try:
    pred = predict(model, message.text, 30)
    bot.reply_to(message, pred)
  except:
    bot.reply_to(message, 'Бот не смог классифицировать отзыв.')

bot.polling(none_stop=True)

Бот запущен

